In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import gc


def save_model(model, checkpointfolder, checkpointname):
    model.save_weights('./' + checkpointfolder + '/' + checkpointname)


def load_model(label_count, checkpointfolder, checkpointname):
    model = create_model(label_count)
    model.load_weights('./' + checkpointfolder + '/' + checkpointname)
    return model

def eval_model_adv():
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Test Accuracy: %.3f' % acc)

    neg_y_test = []
    neg_X_test = []

    pos_y_test = []
    pos_X_test = []

    for idx, row in enumerate(y_test):
      if(y_test[idx] == 0):
        neg_y_test.append(y_test[idx])
        neg_X_test.append(X_test[idx])
        #print(y_test[idx])
        #print(X_test[idx])
        #print(model(X_test[idx].reshape(1, 15730)))
      else:
        pos_y_test.append(y_test[idx])
        pos_X_test.append(X_test[idx])
        #print(y_test[idx])
        #print(X_test[idx])
        #print(model(X_test[idx].reshape(1, 15730)))

    neg_y_test = np.array(neg_y_test)
    neg_X_test = np.array(neg_X_test)
    loss, acc = model.evaluate(neg_X_test, neg_y_test, verbose=1)
    print('Test Accuracy (only negatives): %.3f' % acc)

    pos_y_test = np.array(pos_y_test)
    pos_X_test = np.array(pos_X_test)
    loss, acc = model.evaluate(pos_X_test, pos_y_test, verbose=1)
    print('Test Accuracy (only positives): %.3f' % acc)

In [ ]:
def load_data(type):
    df_ = pd.read_csv('/content/drive/MyDrive/HackathonData/' + type)
    # split into input and output columns
    inputvalues = df_.drop(['label', 'sample_ID'], axis=1)
    outputvalues = df_['label']
    # outputvalues = y_vec
    X, y = inputvalues.values, outputvalues.values
    
    # example of random oversampling to balance the class distribution
    from collections import Counter
    from imblearn.over_sampling import RandomOverSampler
    # define oversampling strategy
    oversample = RandomOverSampler(sampling_strategy='minority')
    # fit and apply the transform
    X, y = oversample.fit_resample(X, y)
    # summarize class distribution
    print(Counter(y))

    y = LabelEncoder().fit_transform(y)

    # split into train and test datasets
    x_train_, x_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.3, random_state=1234) # seed
    #x_train_, x_val_, y_train_, y_val_ = train_test_split(x_train_, y_train_, test_size=0.15, random_state=1234)
    x_val_ = []
    y_val_ = []
    print(x_train_.shape, x_test_.shape, y_train_.shape, y_test_.shape)
    # determine the number of input features
    n_features_ = x_train_.shape[1]
    return df_, x_train_, x_val_, x_test_, y_train_, y_val_, y_test_, n_features_, 1

In [ ]:
def create_model(label_count):
    # define model
    activation = 'relu'
    initializer = tf.keras.initializers.he_normal(seed=1234)
    model_ = Sequential()
    model_.add(Dense(170, activation=activation, kernel_initializer=initializer, input_shape=(n_features,)))
    model_.add(Dense(70, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(13500, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(12000, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(7000, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(3000, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(1, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(200, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(50, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(10, activation=activation, kernel_initializer=initializer))
    #model_.add(Dense(4, activation=activation, kernel_initializer=initializer))
    model_.add(Dense(label_count, activation="sigmoid"))
    # compile the model
    model_.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model_


def fit_model(model, X_train, y_train):
    # fit the model
    #callbacks_list = [PlotLearning()]
    model.fit(X_train, y_train, epochs=42, batch_size=744, verbose=0) #callbacks=callbacks_list


print('_____ ' + 'SpeciesT2D_train' + ' _____')
df, X_train, x_val, X_test, y_train, y_val, y_test, n_features, label_count = load_data('T2D/SpeciesT2D_train.csv')
model = create_model(label_count=label_count)
fit_model(model, X_train, y_train)
save_model(model, "SpeciesT2D", "1x70_42_500_relu")
#model = load_model(label_count, "SpeciesT2D", "1x70_42_500_relu")
eval_model_adv()
gc.collect()

Counter({1: 532, 0: 532})
(744, 15730) (320, 15730) (744,) (320,)
Epoch 1/42
1/1 [==============================] - 0s 432ms/step - loss: 26.7875 - accuracy: 0.4637
Epoch 2/42
1/1 [==============================] - 0s 54ms/step - loss: 35.3380 - accuracy: 0.5282
Epoch 3/42
1/1 [==============================] - 0s 58ms/step - loss: 19.9074 - accuracy: 0.5524
Epoch 4/42
1/1 [==============================] - 0s 50ms/step - loss: 4.1937 - accuracy: 0.7245
Epoch 5/42
1/1 [==============================] - 0s 56ms/step - loss: 4.0637 - accuracy: 0.7433
Epoch 6/42
1/1 [==============================] - 0s 51ms/step - loss: 5.9394 - accuracy: 0.6626
Epoch 7/42
1/1 [==============================] - 0s 50ms/step - loss: 3.8139 - accuracy: 0.7339
Epoch 8/42
1/1 [==============================] - 0s 57ms/step - loss: 1.4644 - accuracy: 0.8616
Epoch 9/42
1/1 [==============================] - 0s 55ms/step - loss: 0.5518 - accuracy: 0.9207
Epoch 10/42
1/1 [==============================] - 0s 49m

3832

In [ ]:
print('_____ ' + 'SpeciesIBD_train' + ' _____')
df, X_train, x_val, X_test, y_train, y_val, y_test, n_features, label_count = load_data('IBD/SpeciesIBD_train.csv')
model = create_model(label_count=label_count)
fit_model(model, X_train, y_train)
#save_model(model, "SpeciesIBD", "1x70_42_500_relu")
#model = load_model(label_count, "SpeciesIBD", "1x70_42_500_relu")
eval_model_adv()
gc.collect()

print('_____ ' + 'SpeciesCAD_train' + ' _____')
df, X_train, x_val, X_test, y_train, y_val, y_test, n_features, label_count = load_data('CAD/SpeciesCAD_train.csv')
model = create_model(label_count=label_count)
fit_model(model, X_train, y_train)
#save_model(model, "SpeciesCAD", "1x70_42_500_relu")
#model = load_model(label_count, "SpeciesCAD", "1x70_42_500_relu")
eval_model_adv()
gc.collect()

print('_____ ' + 'SpeciesCKD_train' + ' _____')
df, X_train, x_val, X_test, y_train, y_val, y_test, n_features, label_count = load_data('CKD/SpeciesCKD_train.csv')
model = create_model(label_count=label_count)
fit_model(model, X_train, y_train)
#save_model(model, "SpeciesCKD", "1x70_42_500_relu")
#model = load_model(label_count, "SpeciesCKD", "1x70_42_500_relu")
eval_model_adv()
gc.collect()